# DES
- DES是使用最广泛的**对称密码**：数据加密标准（Data Encryption Standard）。<br>
- 在对称密码系统（也叫做**私钥密码系统**）中，加密和解密使用**相同的密钥**。<br>
<div>
    <img align=left src="Images/des.png"/>
</div>

## 1. DES原理
### 1.1 基本工作原理
用56位的密钥对64位长的数据块进行16轮加密处理由此得到64位长的密文。
### 1.2 基本工作流程
<div>
    <img align=left src="Images/des-process.png"/>
</div>

## 2. 流程详解
在正式讲解如何把64位明文转换为64位密文之前，我们先讲一下DES加密使用的子密钥是如何生成的。
### 2.1 子密钥生成

子密钥生成的全流程如下：
<div>
    <img align=left src="Images/des-process-key-generate.png"/>
</div>

<div>
    <img align=left src="Images/chi-key-generate-0.png"/>
</div>

#### 2.1.1 输入**64位的密钥**到子密钥算法。

<div>
    <img align=left src="Images/key-generate-1.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.1.2 略去奇偶校验位
把每个字节的第8位作为奇偶校验位,并且略去。DES的密钥由64位减至56位：

<div>
    <img align=left src="Images/key-generate-2.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.1.3  置换选择1
把56位的密钥通过置换选择1，permutation Choice one (PC-1)：

<div>
    <img align=left src="Images/key-generate-3.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.1.4 密钥一分为二
把通过置换选择1的56位密钥，分成2部分（C，D），每部分有28位

<div>
    <img align=left src="Images/key-generate-4.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.1.5 循环左移
每轮中，这两部分分别**循环左移**l位或2位。下表给出了每轮移动的位数：

<div>
    <img align=left src="Images/key-generate-5.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.1.6 置换选择2
置换选择2,permutation choice two (PC-2)，也被称为：**压缩置换**。<br>
- 置换：例如，原第14位在输出时移到了第1位。
- 压缩：第9、18、22、25以及第35、38、43、54均被略去。

<div>
    <img align=left src="Images/key-generate-6.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

经过置换选择2，我们终于实现了：将56位密钥压缩成48位。

## 明文变密文
以上就是子密钥的生成。接下来，我们开始进入正题——**把64位明文转换为64位密文**。<br>

<div>
    <img align=left src="Images/des-process-showtime.png"/>
</div>

如图所示，要把64比特的明文转换成64比特的密文，需要经历：
1. 初始置换
2. 与子密钥K进行16轮迭代
3. 逆初始置换

### 2.2 初始置换
和子密钥的置换选择一样，64位的明文也要经历置换选择。

#### 2.2.1  把明文作为输入
初始置换把**64位**明文作为输入：

<div>
    <img align=left src="Images/IP1.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.2.2  初始置换明文
初始置换把明文按照原有的**64位**进行置换，最后还是**64位**：

<div>
    <img align=left src="Images/IP2.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

#### 2.2.3  一分为二
把通过置换后的64位明文，分成2部分（L，R），每部分有32位

### 2.3 与子密钥K进行16轮迭代
通过2.2.3的一分为二，现在有L、R**各32位**的置换过的明文，我们拿出16轮中的一轮来看看到底这16轮迭代到底做了什么：

<div>
    <img align=left src="Images/des-encryption.jpg"/>
</div>

#### 2.3.1  扩展置换E
扩展置换E：指的是将原本的**32位块**（$R_{i-1}$）扩展到**48位**。<br>
操作：<br>
1. 把32位分成8个4位的块
2. 第 $i$ 块向左，向右各扩展一位，其中左扩展位与第 $i-1$ 块的最右一位相同，右扩展位与第 $i+1$ 块的最左一位相同

<div>
    <img align=left src="Images/EP1.png"/>
    <img align=left src="Images/E-Box.png"/>
</div>

以上图片选自[cybrary](https://www.cybrary.it/0p3n/des-data-encryption-standard/)

将输入的**32位块**（$R_{i-1}$）扩展到**48位**的输出块。这样做的目的其实是实现和**子密钥K**进行**异或运算（XOR）**。(异或运算在**同位**间进行)

#### 2.3.2  与子密钥$K_i$进行异或运算（XOR）
把通过**扩展置换E**扩展的**48位块**与**子密钥K**进行**异或运算（XOR）**<br>
$a \bigoplus b$: 如果a、b两个值不相同，则异或结果为1。如果a、b两个值相同，异或结果为0。

#### 2.3.3  压缩替换S
密钥与扩展分组异或以后，将**48位**的结果进行**代替**运算。替代由**8**个代替盒**（S盒）**完成。
<br>
- 48位的输入 -> 8个6位的分组
- 每一分组对应一个S盒代替操作（共8个）
- 每一个S盒都有6位输入，4位输出

<div>
    <img align=left src="Images/S-Box.gif"/>
</div>

- 8个S盒都不同

<div>
    <img align=left src="Images/8-S-Box.gif"/>
</div>

In [ ]:
s1 = [
[ 14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7 ],
[ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8 ],
[ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0 ],
[ 15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ] ]
 
s2 = [
[ 15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10 ],
[ 3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5 ],
[ 0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15 ],
[ 13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ] ]
 
s3 = [
[ 10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8 ],
[ 13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1 ],
[ 13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7 ],
[ 1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ] ]
 
s4 = [
[ 7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15 ],
[ 13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9 ],
[ 10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4 ],
[ 3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14 ] ]
 
s5 = [
[ 2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9 ],
[ 14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6 ],
[ 4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14 ],
[ 11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ] ]
 
s6 = [
[ 12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11 ],
[ 10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8 ],
[ 9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6 ],
[ 4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13 ] ]
 
s7 = [
[ 4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1 ],
[ 13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6 ],
[ 1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2 ],
[ 6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12 ] ]
 
s8 = [
[ 13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7 ],
[ 1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2 ],
[ 7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8 ],
[ 2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11 ] ]

- 每个S盒是一个4行（0到3）、16列的表（0到15）。S盒的6bit输入确定了其对应的输出在哪一行哪一列。以$S_6$盒为例：

<div>
    <img align=left src="Images/S-Box-6.png"/>
</div>

假定将S盒的**6bit**的输入标记为b1、b2、b3、b4、b5、b6，则：
- b1和b6对应行：0到3
- b2到b5对应列：0到15

##### example
假如$S_6$盒子要处理的**6bit**数据为：110011
- b1,b2,b3,b4,b,b6 = 1,1,0,0,1,1
- 负责行的b1和b6为：$11_2 = 3_{10}$
- 负责列的b2,b3,b4, b5为：$1001_2 = 9_{10}$
- 由此可见，置换的是3行9列的：$14_{10} = 1110_2$
- 110011 -> $S_6$置换 -> 1110

<div>
    <img align=left src="Images/S-Box-6-new.png"/>
</div>

- 48位块 -> S盒 -> 32位块

将**48位**压缩到**32位**。这样做的目的其实是实现和$L$**（32位块）**进行**异或运算（XOR）**。

#### 2.3.4  P盒置换

<div>
    <img align=left src="Images/P-Box.png"/>
</div>

P置换使得一个S盒的输出对下一轮多个S盒产生影响，形成**雪崩效应**：明文或密钥的一点小的变动都引起密文的较大变化。<br>
将P盒置换的结果与最初的64位分组的左半部分异或，接着开始另一轮。

#### 2.3.5  与初始置换之后的左边一半明文$L_i$进行异或运算（XOR）
$a \bigoplus b$: 如果a、b两个值不相同，则异或结果为1。如果a、b两个值相同，异或结果为0。

###   2.4 逆初始置换
再将64位的明文进行**逆初始置换**：

<div>
    <img align=left src="Images/IP3.png"/>
</div>

<div>
    <img align=left src="Images/IP-IP-1.png"/>
</div>

In [ ]:
ip = [  58, 50, 42, 34, 26, 18, 10, 2, 60, 52, 44, 36, 28, 20, 12, 4,
		62, 54, 46, 38, 30, 22, 14, 6, 64, 56, 48, 40, 32, 24, 16, 8,
		57, 49, 41, 33, 25, 17,  9, 1, 59, 51, 43, 35, 27, 19, 11, 3,
		61, 53, 45, 37, 29, 21, 13, 5, 63, 55, 47, 39, 31, 23, 15, 7 ]
 
_ip = [ 40, 8, 48, 16, 56, 24, 64, 32, 39, 7,47, 15, 55, 23, 63, 31,
		38, 6, 46, 14, 54, 22, 62, 30, 37, 5, 45,13, 53, 21, 61, 29,
		36, 4, 44, 12, 52, 20, 60, 28, 35, 3, 43, 11,51, 19, 59, 27,
		34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25 ]

###   2.5 解密
- 加密和解密可使用相同的算法。
- DES解密结构与其加密结构是对称相似的，使得能用相同的函数来加密或解密每个分组。
- 二者的唯一不同之处是密钥的次序相反。

<div>
    <img align=left src="Images/IP-1-IP.png"/>
</div>